In [1]:
import tensorflow as tf
import datetime

d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
(train_image,train_labels),(test_image,test_labels) = tf.keras.datasets.mnist.load_data()

In [4]:
train_image.shape

(60000, 28, 28)

In [5]:
train_image = tf.expand_dims(train_image,-1)
test_image = tf.expand_dims(test_image,-1)

In [7]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [8]:
train_image = tf.cast(train_image/255, tf.float32)
test_image =tf.cast(test_image/255, tf.float32)

In [9]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image,test_labels))

In [12]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [13]:
dataset = dataset.repeat().shuffle(60000).batch(128)
test_dataset = test_dataset.repeat().batch(128)

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [32]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [33]:
import os
log_dir = os.path.join('logs',datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

In [34]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir,histogram_freq=1)

In [35]:
#控制学习速率
def lr_sche(epoch):
    learning_rate = 0.2
    if epoch > 5:
        learning_rate = 0.02
    if epoch > 10:
        learning_rate = 0.01
    if epoch > 20:
        learning_rate = 0.005
    tf.summary.scalar('learning_rate', data = learning_rate, step = epoch)
    return learning_rate

In [36]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [37]:
file_writer = tf.summary.create_file_writer(log_dir + '/lr')

In [38]:
file_writer.set_as_default()

In [39]:
model.fit(dataset,
         epochs=25,
         steps_per_epoch=60000//128,
         validation_data=test_dataset,
         validation_steps=10000//128,
         callbacks=[tensorboard_callback,lr_callback])

Epoch 1/25
468/468 [==============================] - 4s 9ms/step - loss: 2.3867 - accuracy: 0.1043 - val_loss: 2.3066 - val_accuracy: 0.1010
Epoch 2/25
468/468 [==============================] - 4s 8ms/step - loss: 2.3121 - accuracy: 0.1025 - val_loss: 2.3141 - val_accuracy: 0.0891
Epoch 3/25
468/468 [==============================] - 4s 8ms/step - loss: 2.3128 - accuracy: 0.1044 - val_loss: 2.3101 - val_accuracy: 0.0981
Epoch 4/25
468/468 [==============================] - 4s 8ms/step - loss: 2.3123 - accuracy: 0.1025 - val_loss: 2.3093 - val_accuracy: 0.1135
Epoch 5/25
468/468 [==============================] - 4s 8ms/step - loss: 2.3123 - accuracy: 0.1029 - val_loss: 2.3132 - val_accuracy: 0.1010
Epoch 6/25
468/468 [==============================] - 4s 8ms/step - loss: 2.3139 - accuracy: 0.1021 - val_loss: 2.3077 - val_accuracy: 0.1135
Epoch 7/25
468/468 [==============================] - 4s 8ms/step - loss: 2.3023 - accuracy: 0.1089 - val_loss: 2.3019 - val_accuracy: 0.1135
Epoch 

KeyboardInterrupt: 

In [22]:
%load_ext tensorboard
%matplotlib inline


The tensorboard module is not an IPython extension.


In [23]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


# 自定义训练中使用Tensorboard

In [43]:
optimizer = tf.keras.optimizers.Adam()

In [44]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [45]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)

In [46]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [47]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [48]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels,pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [49]:
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = 'logs/gradient_tape' + current_time + '/train'
test_log_dir = 'logs/gradient_tape' + current_time + '/test'
train_writer = tf.summary.create_file_writer(train_log_dir)
test_writer = tf.summary.create_file_writer(test_log_dir)

In [51]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        
        with train_writer.as_default():
            tf.summary.scalar('loss',train_loss.result(),step=epoch)
            tf.summary.scalar('acc',train_accuracy.result(),step=epoch)
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
            print('*', end='')
        
        with test_writer.as_default():
            tf.summary.scalar('loss',test_loss.result(),step=epoch)
            tf.summary.scalar('acc',test_accuracy.result(),step=epoch)
            
        template = 'Epoch {},Loss:{} , Accuracy:{}, Test Loss:{}, Test Accuracy:{}'
        print(template.format(epoch+1,
                             train_loss.result(),
                             train_accuracy.result(),
                             test_loss.result(),
                             test_accuracy.result()))

In [ ]:
train()